In [19]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
import json
from collections import defaultdict

# Conexión al clúster de Cassandra con los nombres de host de tus nodos
cluster = Cluster(['127.0.0.1'], port=9042)#, auth_provider=auth_provider)

# Crear una sesión
session = cluster.connect()

# Seleccionar el keyspace (si ya tienes uno creado)
session.set_keyspace('videojuego')

## Lectura hall of fame

In [22]:
def get_hall_of_fame(country: str):
    # Establecer consistencia más fuerte
    # Para asegurar la máxima consistencia usamos ALL. Todas las réplicas tendrán que responder a la query
    session.default_consistency_level = ConsistencyLevel.ALL

    # Query 
    query = """
        SELECT Mazmorra_id, Nombre_mazmorra, Email, Nombre_usuario, Tiempo, Fecha 
        FROM Hall_of_fame 
        WHERE Pais = %s
        ALLOW FILTERING;
    """
    
    rows = session.execute(query, (country, ))
    
    # Organizar datos en un diccionario agrupado por Mazmorra_id
    dungeon_dict = defaultdict(lambda: {"dungeon_id": 0, "dungeon_name": "", "Top_5": []})

    for row in rows:
        dungeon_id = row.mazmorra_id
        dungeon_dict[dungeon_id]["dungeon_id"] = dungeon_id
        dungeon_dict[dungeon_id]["dungeon_name"] = row.nombre_mazmorra
        dungeon_dict[dungeon_id]["Top_5"].append({
            "email": row.email,
            "user_name": row.nombre_usuario,
            "time_minutes": row.tiempo,
            "date": row.fecha
        })
    
    result = json.dumps(list(dungeon_dict.values()), indent=4) 
    return result


In [23]:
# Ejemplo de uso
country = "ja_JP"
top_players = get_hall_of_fame(country)
print(top_players)


[
    {
        "dungeon_id": 2,
        "dungeon_name": "Shadysparth, Hobble of the Wandering Stoners",
        "Top_5": [
            {
                "email": "chiyo96@example.net",
                "user_name": "ryohei00",
                "time_minutes": 1.0,
                "date": "2019-06-11 10:45:51"
            },
            {
                "email": "bnakajima@example.com",
                "user_name": "yutakobayashi",
                "time_minutes": 1.0,
                "date": "2018-04-03 20:46:35"
            },
            {
                "email": "atsushi94@example.org",
                "user_name": "maifujii",
                "time_minutes": 1.0,
                "date": "2021-12-21 03:53:21"
            },
            {
                "email": "asukasuzuki@example.net",
                "user_name": "gfujiwara",
                "time_minutes": 1.0,
                "date": "2022-08-16 03:24:18"
            },
            {
                "email": "bwatanabe@example.

/tmp/ipykernel_411081/1282580080.py:4: DeprecationWarning: Setting the consistency level at the session level will be removed in 4.0. Consider using execution profiles and setting the desired consistency level to the EXEC_PROFILE_DEFAULT profile.
  session.default_consistency_level = ConsistencyLevel.ALL


## Lectura Statistics

In [13]:
def get_user_statistics(user_email: str, dungeon_id: int):
    # Establecer consistencia más fuerte
    session.default_consistency_level = ConsistencyLevel.ALL
    
    # Query 
    query = """
        SELECT Tiempo, Fecha 
        FROM Statistic 
        WHERE Email = %s AND Mazmorra_id = %s;
    """
    
    rows = session.execute(query, (user_email, dungeon_id))
    
    # converitr a lista de diccionarios
    result = [{"time_minute": row.tiempo, "date": row.fecha} for row in rows]
    
    # Convertir a JSON
    return json.dumps(result, indent=4)


In [21]:
# Ejemplo de uso
user_email = 'aaoki@example.com'
dungeon_id = 0
statistics_json = get_user_statistics(user_email, dungeon_id)
print(statistics_json)


[
    {
        "time_minute": 0.10000000149011612,
        "date": "2020-04-18 04:46:20"
    },
    {
        "time_minute": 21.0,
        "date": "2022-08-14 22:37:30"
    },
    {
        "time_minute": 29.0,
        "date": "2021-04-16 13:34:41"
    },
    {
        "time_minute": 40.0,
        "date": "2022-10-12 00:37:08"
    },
    {
        "time_minute": 42.0,
        "date": "2022-09-01 03:51:33"
    },
    {
        "time_minute": 46.0,
        "date": "2021-01-22 23:40:44"
    },
    {
        "time_minute": 47.0,
        "date": "2021-12-10 09:51:10"
    }
]


/tmp/ipykernel_411081/2746293061.py:3: DeprecationWarning: Setting the consistency level at the session level will be removed in 4.0. Consider using execution profiles and setting the desired consistency level to the EXEC_PROFILE_DEFAULT profile.
  session.default_consistency_level = ConsistencyLevel.ALL


## Lectura Top horde

In [15]:
def get_top_horde(country: str, event_id: int, k: int):
    # Reducimos consistencia para asegurar velocidad en las lecturas
    session.default_consistency_level = ConsistencyLevel.LOCAL_ONE

    # Query optimizada
    query = """
        SELECT Email, Nombre_usuario, N_killed 
        FROM Top_horde 
        WHERE Evento_id = %s AND Pais = %s 
        LIMIT %s;
    """
    
    rows = session.execute(query, (event_id, country, k))
    
    # Convertir a lista de diccionarios
    result = [
        {
            "posicion": idx + 1,  # Asignamos un ranking (opcional)
            "user_name": row.nombre_usuario,
            "email": row.email,
            "n_killed": row.n_killed
        }
        for idx, row in enumerate(rows)
    ]

    # Convertir a JSON
    return json.dumps(result, indent=4)


In [ ]:
# Ejemplo de uso
country = "ja_JP"
event_id = 2
k = 3
top_horde_json = get_top_horde(country, event_id, k)
print(top_horde_json)


[
    {
        "posicion": 1,
        "user_name": "ukobayashi",
        "email": "jyamashita@example.org",
        "n_killed": 22
    },
    {
        "posicion": 2,
        "user_name": "ltanaka",
        "email": "ymurakami@example.com",
        "n_killed": 22
    },
    {
        "posicion": 3,
        "user_name": "gfukuda",
        "email": "inoueyosuke@example.com",
        "n_killed": 21
    }
]


/tmp/ipykernel_411081/235877665.py:3: DeprecationWarning: Setting the consistency level at the session level will be removed in 4.0. Consider using execution profiles and setting the desired consistency level to the EXEC_PROFILE_DEFAULT profile.
  session.default_consistency_level = ConsistencyLevel.LOCAL_ONE


## Cerramos conexión

In [17]:
# Cerrar la conexión
cluster.shutdown()